# Quiz

## 1

(Single Choice) In horizontal federated learning (HFL), participating organizations primarily differ in 

a) the set of features they hold 

b) the set of samples they hold

c) the training algorithm 

d) the encryption scheme



## 2
Consider these two scenarios:

1. Several hospitals in different cities each keep the same set of clinical variables (age, sex, lab tests) but for different patients.
2. A commercial bank and an e-commerce platform both serve many of the same customers: the bank owns account balances and credit scores, while the retailer owns browsing and purchase histories.

which scenario is suitable of HFL and which is suitable for VFL , and why 



## 3
Explain briefly why data heterogeneity across clients can slow convergence in horizontal FL

# Coding

We try to implement a single-model FL example 

In [1]:
"""
Server-based (centralized) Federated Learning demo
--------------------------------------------------
• Synthetic linear-regression data for N clients
• One global model (simple Linear without bias)
• FedAvg aggregation on the server
"""

import torch, torch.nn as nn, torch.optim as optim
from copy import deepcopy

# ------------------------------ hyper-parameters ------------------------------
NUM_CLIENTS      = 3        # number of edge devices
SAMPLES_PER_CL   = 100      # local data size
N_FEATURES       = 5        # dimensionality of x
ROUNDS           = 20       # FedAvg communication rounds
LOCAL_EPOCHS     = 1        # SGD passes per client per round
LR               = 0.1      # local learning rate
SEED             = 0        # reproducibility
# ------------------------------------------------------------------------------

torch.manual_seed(SEED)

# ------------------------------ synthetic data -------------------------------
true_w = torch.randn(N_FEATURES, 1)           # unknown ground-truth weights

clients_data = []
for _ in range(NUM_CLIENTS):
    X = torch.randn(SAMPLES_PER_CL, N_FEATURES)
    y = X @ true_w + 0.1 * torch.randn(SAMPLES_PER_CL, 1)  # noisy labels
    clients_data.append((X, y))
# ------------------------------------------------------------------------------

# ------------------------------ model definition -----------------------------
class LinearModel(nn.Module):
    """Single-layer linear regression w/o bias to keep things simple."""
    def __init__(self, d: int):
        super().__init__()
        self.w = nn.Parameter(torch.zeros(d, 1))

    def forward(self, x):
        return x @ self.w
# ------------------------------------------------------------------------------

# ------------------------------ FedAvg helper --------------------------------
def fed_avg(state_dicts, sizes):
    """Weighted (by sample count) parameter averaging."""
    new_state = deepcopy(state_dicts[0])
    for k in new_state:
        new_state[k].data.zero_()
    total = float(sum(sizes))
    for st, n in zip(state_dicts, sizes):
        for k in st:
            new_state[k] += st[k] * (n / total)
    return new_state
# ------------------------------------------------------------------------------

# ------------------------------ training loop --------------------------------
global_model = LinearModel(N_FEATURES)        # w^(0)

for rnd in range(ROUNDS):
    client_states, sizes = [], []

    # ----------- step 1: server → clients (broadcast w^(k)) -----------
    for X, y in clients_data:
        # copy global model to the client
        client = LinearModel(N_FEATURES)
        client.load_state_dict(global_model.state_dict())

        # ----------- step 2: local update (compute w^(i,k)) -----------
        opt = optim.SGD(client.parameters(), lr=LR)
        loss_fn = nn.MSELoss()

        for _ in range(LOCAL_EPOCHS):
            opt.zero_grad()
            loss = loss_fn(client(X), y)
            loss.backward()
            opt.step()

        # send update back to server
        client_states.append(client.state_dict())
        sizes.append(len(X))

    # ----------- step 3: server aggregation (FedAvg) ------------------
    new_state = fed_avg(client_states, sizes)
    global_model.load_state_dict(new_state)   # w^(k+1)

    # optional: monitor global loss
    with torch.no_grad():
        mse, total = 0.0, 0
        for X, y in clients_data:
            mse += ((global_model(X) - y) ** 2).sum().item()
            total += len(X)
        print(f"Round {rnd+1:02d}: MSE = {mse / total:.4f}")

# ------------------------------ results --------------------------------------
print("\nTrue weights : ", true_w.squeeze().tolist())
print("Learned weights:", global_model.w.squeeze().tolist())


Round 01: MSE = 5.3397
Round 02: MSE = 3.5184
Round 03: MSE = 2.3250
Round 04: MSE = 1.5410
Round 05: MSE = 1.0248
Round 06: MSE = 0.6841
Round 07: MSE = 0.4589
Round 08: MSE = 0.3096
Round 09: MSE = 0.2105
Round 10: MSE = 0.1445
Round 11: MSE = 0.1005
Round 12: MSE = 0.0712
Round 13: MSE = 0.0515
Round 14: MSE = 0.0383
Round 15: MSE = 0.0295
Round 16: MSE = 0.0235
Round 17: MSE = 0.0195
Round 18: MSE = 0.0168
Round 19: MSE = 0.0149
Round 20: MSE = 0.0137

True weights :  [1.5409960746765137, -0.293428897857666, -2.1787893772125244, 0.5684312582015991, -1.0845223665237427]
Learned weights: [1.5136923789978027, -0.2985497713088989, -2.133829355239868, 0.5473030805587769, -1.0536893606185913]
